<a href="https://colab.research.google.com/github/arezoosh/PhD-Course1/blob/master/trumpfake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/DeepLearnI/trump_tweet_classifier/master/code/tweet_labels.csv')

In [20]:
data.tweet.head()

0    To every one of the HEROES we recognized today...
1    We have been serving as policemen in Afghanist...
2    ...the Economy, where there is NO Recession, m...
3    ...the look of turmoil in the White House, of ...
4    A lot of Fake News is being reported that I ov...
Name: tweet, dtype: object

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15348 entries, 0 to 15347
Data columns (total 2 columns):
tweet     15348 non-null object
labels    15348 non-null bool
dtypes: bool(1), object(1)
memory usage: 135.0+ KB


In [0]:
X = data.loc[:,'tweet']
Y = data.loc[:,'labels']

In [12]:
X.head()
Y.head()

0    True
1    True
2    True
3    True
4    True
Name: labels, dtype: bool

In [0]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [0]:
y_enc = encoder.fit_transform(Y)

In [0]:
 X_train, X_test, y_train, y_test = train_test_split( X, y_enc, test_size=0.2, random_state=42)

In [23]:
X_train.head()

7573     We are going to defend our industry & create a...
573      What the Market wanted to hear from Jay Powell...
6177     Bernie Sanders supporters have every right to ...
14844    .@SteveHilton is on @FoxNews, on the air. He i...
14350    President @realDonaldTrump has signed (prohibi...
Name: tweet, dtype: object

In [0]:
import re

pattern = re.compile('<br /><br />')

In [0]:
#not sure this part is necassary
train_df = X_train.apply(lambda text: pattern.subn(' ', text)[0])
test_df = X_test.apply(lambda text: pattern.subn(' ', text)[0])

In [29]:
train_df.head()

7573     We are going to defend our industry & create a...
573      What the Market wanted to hear from Jay Powell...
6177     Bernie Sanders supporters have every right to ...
14844    .@SteveHilton is on @FoxNews, on the air. He i...
14350    President @realDonaldTrump has signed (prohibi...
Name: tweet, dtype: object

In [30]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
!pip install -qq -U gensim

     |████████████████████████████████| 24.2MB 98kB/s 


In [0]:
from gensim.corpora.dictionary import Dictionary

In [0]:
from gensim.models.tfidfmodel import TfidfModel

In [0]:
from gensim.models.lsimodel import LsiModel

In [0]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [0]:
# Tokenize our texts and remove stopwords, also kick out numbers, lower everything

train_tokens = train_df.map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])

In [0]:
test_tokens = test_df.map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])

In [0]:
dictionary = Dictionary(train_tokens)

In [48]:
print(dictionary)


Dictionary(2697 unique tokens: ['americafirst', 'american', 'create', 'defend', 'field']...)


In [50]:
dictionary.filter_extremes(no_below = 10, no_above=0.4)
train_tokens

7573     [we, going, defend, industry, create, level, p...
573      [what, market, wanted, hear, jay, powell, fede...
6177     [bernie, sanders, supporters, every, right, ap...
14844    [stevehilton, foxnews, air, he, fine, guest, g...
14350    [president, realdonaldtrump, signed, prohibito...
                               ...                        
5191     [great, meeting, prime, minister, abe, japan, ...
13418                                       [show, thread]
5390                           [thank, hosting, magahttps]
860                    [mueller, s, partisan, witch, hunt]
7270     [icymi, announcement, air, traffic, control, i...
Name: tweet, Length: 12278, dtype: object

In [0]:
train_corpus = [dictionary.doc2bow(doc) for doc in train_tokens]
test_corpus = [dictionary.doc2bow(doc) for doc in test_tokens]


In [0]:
from gensim.matutils import corpus2csc, corpus2dense

In [0]:

X_train = corpus2csc(train_corpus)
X_test = corpus2csc(test_corpus)

In [56]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train.T, y_train)

model.score(X_test.T, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7055374592833876

In [0]:
tfidf = TfidfModel(train_corpus)

In [0]:
train_corpus_tfidf = tfidf[train_corpus]
test_corpus_tfidf = tfidf[test_corpus]

In [0]:
X_train = corpus2csc(train_corpus_tfidf)
X_test = corpus2csc(test_corpus_tfidf)

In [63]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train.T, y_train)

model.score(X_test.T, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.709771986970684

In [0]:
from gensim.models.lsimodel import LsiModel

In [0]:
# Training the LSI model
model_lsi = LsiModel(train_corpus_tfidf, num_topics = 300, id2word=dictionary)

In [0]:
# Generating the corpus train & test

train_corpus_lsi = model_lsi[train_corpus_tfidf]
test_corpus_lsi = model_lsi[test_corpus_tfidf]

In [0]:
# turn into matrix
train_lsi_corpus = corpus2dense(train_corpus_lsi, 300 )

test_lsi_corpus = corpus2dense(test_corpus_lsi, 300)

In [68]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(train_lsi_corpus.T, y_train)

model.score(test_lsi_corpus.T, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6716612377850163

In [0]:
# Load the MatrixSimilarity
from gensim.similarities import MatrixSimilarity

# Create the document-topic-matrix
document_topic_matrix_train = MatrixSimilarity(train_corpus_lsi)

# this will create you a document - document similarity matrix (you could import it as a network...)
document_topic_matrix_train_ix = document_topic_matrix_train.index

# Same for test-set
document_topic_matrix_test = MatrixSimilarity(test_corpus_lsi)
document_topic_matrix_test_ix = document_topic_matrix_test.index

In [82]:
doc = "I use both iPhone & Samsung. If Apple doesn't give info to authorities on the terrorists I'll only be using Samsung until they give info"

vec_bow = dictionary.doc2bow(doc.lower().split()) # convert to bag of words
vec_tfidf = tfidf[vec_bow] # convert to tfidf
vec_lsi = model_lsi[vec_tfidf]  # convert the query to LSI space

print(len(vec_lsi))
print(vec_lsi[:10])

300
[(0, 0.09572592901988965), (1, -0.028168842805818675), (2, 0.0172924581064519), (3, -0.014946780638606903), (4, 0.0020966095148325144), (5, -0.002499010009224768), (6, -0.01757893536566757), (7, -0.002337395799302267), (8, 0.01949380077495537), (9, 0.011204962840145626)]


In [83]:
sims = document_topic_matrix_train[vec_lsi]

sims = sorted(enumerate(sims), key=lambda item: -item[1])
for i, s in sims[:10]: #ten most similar texts
    print(s, train_df[i])

0.49509013 “79% of these decisions have been overturned in the 9th Circuit.” @FoxNews A terrible, costly and dangerous disgrace. It has become a dumping ground for certain lawyers looking for easy wins and delays. Much talk over dividing up the 9th Circuit into 2 or 3 Circuits. Too big!
0.4584 Democrat Rep. Tlaib is being slammed for her horrible and highly insensitive statement on the Holocaust. She obviously has tremendous hatred of Israel and the Jewish people. Can you imagine what would happen if I ever said what she said, and says?
0.45589748 Going to play golf right now with @AbeShinzo. Japan loves the game. Tremendous fans of @JackNicklaus, @TigerWoods, and @PhilMickelson — I said what about @GaryPlayer, they said we love Gary too!
0.4546603 Thank you to Chairman Kim Jong Un for keeping your faith in our great U.S. When I took office, your airplane, probably the biggest in the world, was just coming into a landing pattern in Japan. A few months later, because of your patience an